In [1]:
import numpy as np
import pandas as pd
import json
import nltk
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import re
import math
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
def load_SNLI(path:str):
    with open(path, 'r') as f:
        # Read all lines in the file and parse them as JSON
        data = [json.loads(line.strip()) for line in f]
    return data

def load_SNLI_as_pandas(path:str):
    df = pd.read_json(path, lines=True)
    return df

In [3]:
#load the training data into a variable for future use
path_data = "SNLI_data/snli_1.0_train.jsonl"
snli_data = load_SNLI(path=path_data)

In [4]:
snli_data_pd = load_SNLI_as_pandas(path=path_data)

In [5]:
snli_data_pd.head(10)

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],3416050480.jpg#4,neutral,3416050480.jpg#4r1n,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
1,[contradiction],3416050480.jpg#4,contradiction,3416050480.jpg#4r1c,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is at a diner, ordering an omelette.",( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
2,[entailment],3416050480.jpg#4,entailment,3416050480.jpg#4r1e,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is outdoors, on a horse.","( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
3,[neutral],2267923837.jpg#2,neutral,2267923837.jpg#2r1n,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,They are smiling at their parents,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...
4,[entailment],2267923837.jpg#2,entailment,2267923837.jpg#2r1e,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,There are children present,( There ( ( are children ) present ) ),(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...
5,[contradiction],2267923837.jpg#2,contradiction,2267923837.jpg#2r1c,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,The kids are frowning,( ( The kids ) ( are frowning ) ),(ROOT (S (NP (DT The) (NNS kids)) (VP (VBP are...
6,[contradiction],3691670743.jpg#0,contradiction,3691670743.jpg#0r1c,A boy is jumping on skateboard in the middle o...,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,The boy skates down the sidewalk.,( ( The boy ) ( ( ( skates down ) ( the sidewa...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ skate...
7,[entailment],3691670743.jpg#0,entailment,3691670743.jpg#0r1e,A boy is jumping on skateboard in the middle o...,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,The boy does a skateboarding trick.,( ( The boy ) ( ( does ( a ( skateboarding tri...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ does)...
8,[neutral],3691670743.jpg#0,neutral,3691670743.jpg#0r1n,A boy is jumping on skateboard in the middle o...,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,The boy is wearing safety equipment.,( ( The boy ) ( ( is ( wearing ( safety equipm...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ is) (...
9,[neutral],4804607632.jpg#0,neutral,4804607632.jpg#0r1n,An older man sits with his orange juice at a s...,( ( An ( older man ) ) ( ( ( sits ( with ( ( h...,(ROOT (S (NP (DT An) (JJR older) (NN man)) (VP...,An older man drinks his juice as he waits for ...,( ( An ( older man ) ) ( ( ( drinks ( his juic...,(ROOT (S (NP (DT An) (JJR older) (NN man)) (VP...


### Preprocessing functions

In [6]:
# Here to implement all functions for preprocessing the given data
def remove_duplicates(df:pd.DataFrame, subset:list[str]) -> pd.DataFrame:
    df_preprocessed = df.drop_duplicates(subset=subset)

    return df_preprocessed

def lower_tokenize_punct(lst:list[str]):
    stopwords_english = set(stopwords.words("english"))
    preprocessed_lst = []
    for sent in lst:
        lowercased_sent = sent.lower() # lowercase
        tokens_of_sent = word_tokenize(lowercased_sent) # tokenize
        clean_punctuation = [w for w in tokens_of_sent if w not in stopwords_english and w not in string.punctuation] # handle stopwords and punctuation

        preprocessed_lst.append(clean_punctuation)
    return preprocessed_lst
    
#lowercase, remove stopwords, tokenize --> Add new columns
def preprocess_dataF(df: pd.DataFrame) -> pd.DataFrame:
    df_out = df.copy(deep=True)
    premises = df["sentence1"].values
    hypotheses = df["sentence2"].values

    preprocessed_premises = lower_tokenize_punct(premises)
    preprocessed_hypotheses = lower_tokenize_punct(hypotheses)

    df_out["sentence1_preprocessed"] = preprocessed_premises
    df_out["sentence2_preprocessed"] = preprocessed_hypotheses

    return df_out

### Helpful Functions

In [7]:
def count_words_dict(df: pd.DataFrame) -> dict:
    out_count_dict = nltk.defaultdict(int)
    premises = df["sentence1_preprocessed"].values
    hypotheses = df["sentence2_preprocessed"].values

    merge = premises + hypotheses

    for sent in merge:
        for word in sent:
            out_count_dict[word] += 1
    return out_count_dict

def sort_dict(d: dict, reverse=True) -> dict:
    d_out = sorted(d.items(), key=lambda x: x[1] , reverse=reverse)
    return dict(d_out)

def remove_less_than_n_dict(d: dict, n:int) -> dict:
    new_dict = {k: v for k, v in d.items() if v >=n}
    return new_dict

In [8]:
print(f"Count of ALL samples : {len(snli_data_pd)}")

Count of ALL samples : 550152


In [9]:
subset_distinct = ["sentence1", "sentence2"]
snli_data_distinct = remove_duplicates(df=snli_data_pd, subset=subset_distinct)

In [10]:
print(f"Count of distinct samples : {len(snli_data_distinct)}")

Count of distinct samples : 549526


In [11]:
a = []
b = [1, 2, 3]
c = [2, 3, 555]
a.append(b)
a.append(c)
a

[[1, 2, 3], [2, 3, 555]]

In [12]:
snli_data_pd_preprocessed = preprocess_dataF(snli_data_distinct)

In [13]:
snli_data_pd_preprocessed.columns

Index(['annotator_labels', 'captionID', 'gold_label', 'pairID', 'sentence1',
       'sentence1_binary_parse', 'sentence1_parse', 'sentence2',
       'sentence2_binary_parse', 'sentence2_parse', 'sentence1_preprocessed',
       'sentence2_preprocessed'],
      dtype='object')

In [14]:
snli_data_pd_preprocessed.head()

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse,sentence1_preprocessed,sentence2_preprocessed
0,[neutral],3416050480.jpg#4,neutral,3416050480.jpg#4r1n,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,"[person, horse, jumps, broken, airplane]","[person, training, horse, competition]"
1,[contradiction],3416050480.jpg#4,contradiction,3416050480.jpg#4r1c,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is at a diner, ordering an omelette.",( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,"[person, horse, jumps, broken, airplane]","[person, diner, ordering, omelette]"
2,[entailment],3416050480.jpg#4,entailment,3416050480.jpg#4r1e,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is outdoors, on a horse.","( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,"[person, horse, jumps, broken, airplane]","[person, outdoors, horse]"
3,[neutral],2267923837.jpg#2,neutral,2267923837.jpg#2r1n,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,They are smiling at their parents,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...,"[children, smiling, waving, camera]","[smiling, parents]"
4,[entailment],2267923837.jpg#2,entailment,2267923837.jpg#2r1e,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,There are children present,( There ( ( are children ) present ) ),(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...,"[children, smiling, waving, camera]","[children, present]"


In [15]:
snli_data_pd_preprocessed["annotator_labels"][1]

['contradiction']

In [16]:
c_dict = count_words_dict(snli_data_pd_preprocessed)

In [17]:
c_dict, len(c_dict)

(defaultdict(int,
             {'person': 35987,
              'horse': 7221,
              'jumps': 6628,
              'broken': 736,
              'airplane': 1219,
              'training': 734,
              'competition': 2594,
              'diner': 253,
              'ordering': 260,
              'omelette': 2,
              'outdoors': 9109,
              'children': 26545,
              'smiling': 8635,
              'waving': 1370,
              'camera': 11371,
              'parents': 1024,
              'present': 851,
              'kids': 10748,
              'frowning': 312,
              'boy': 58262,
              'jumping': 13405,
              'skateboard': 4720,
              'middle': 4580,
              'red': 43302,
              'bridge': 3216,
              'skates': 534,
              'sidewalk': 12819,
              'skateboarding': 1981,
              'trick': 3313,
              'wearing': 80985,
              'safety': 1904,
              'equipment': 2

In [18]:
a = [[1, 2], [2, 5, 6]]
b = [[1, 4, 5, 6]]
a + b 

[[1, 2], [2, 5, 6], [1, 4, 5, 6]]

In [19]:
sorted_dict = sort_dict(c_dict)

In [20]:
sorted_dict_10 = remove_less_than_n_dict(d=sorted_dict, n=10)

In [21]:
len(sorted_dict_10)

12398

In [22]:
premises11 = list(snli_data_pd_preprocessed["sentence1_preprocessed"].values)
hypotheses11 = list(snli_data_pd_preprocessed["sentence2_preprocessed"].values)


merge_both_as_tuple = list(zip(premises11, hypotheses11))
print(merge_both_as_tuple[:10])

[(['person', 'horse', 'jumps', 'broken', 'airplane'], ['person', 'training', 'horse', 'competition']), (['person', 'horse', 'jumps', 'broken', 'airplane'], ['person', 'diner', 'ordering', 'omelette']), (['person', 'horse', 'jumps', 'broken', 'airplane'], ['person', 'outdoors', 'horse']), (['children', 'smiling', 'waving', 'camera'], ['smiling', 'parents']), (['children', 'smiling', 'waving', 'camera'], ['children', 'present']), (['children', 'smiling', 'waving', 'camera'], ['kids', 'frowning']), (['boy', 'jumping', 'skateboard', 'middle', 'red', 'bridge'], ['boy', 'skates', 'sidewalk']), (['boy', 'jumping', 'skateboard', 'middle', 'red', 'bridge'], ['boy', 'skateboarding', 'trick']), (['boy', 'jumping', 'skateboard', 'middle', 'red', 'bridge'], ['boy', 'wearing', 'safety', 'equipment']), (['older', 'man', 'sits', 'orange', 'juice', 'small', 'table', 'coffee', 'shop', 'employees', 'bright', 'colored', 'shirts', 'smile', 'background'], ['older', 'man', 'drinks', 'juice', 'waits', 'daught

In [23]:
def co_occurence_two_words(df: pd.DataFrame, w_i, w_j) -> int:
    premises = list(df["sentence1_preprocessed"].values)
    hypotheses = list(df["sentence2_preprocessed"].values)
    count = 0

    merge_both_as_tuple = list(zip(premises, hypotheses))
    for p, h in merge_both_as_tuple:
        if w_i in p and w_j in p:
            count += 1
        if w_i in h and w_j in h:
            count += 1
    return count

def calculate_pmi(df:pd.DataFrame, N: int, counts:dict, w_i, w_j) -> float:
    co_occ_value = co_occurence_two_words(df=df, w_i= w_i, w_j=w_j)
    count_wi = counts[w_i]
    count_wj = counts[w_j]

    numerator = float(N * co_occ_value)
    denominator = float(count_wi * count_wj)
    if numerator == 0:
        return 0
    
    return math.log2(numerator/denominator)

def tool(df: pd.DataFrame, N: int, counts:dict, w:str) -> dict:
    out_dict = nltk.defaultdict(int)
    counter = 0
    for k,v in counts.items():
        pmi = calculate_pmi(df=df, N=N, counts=counts, w_i=w, w_j=k)
        out_dict[k] = pmi
        print(f"iter {counter} : {k}, PMI: {pmi}")
        counter += 1
        
    #rank by PMI
    return dict(sorted(out_dict.items(), key=lambda x: x[1], reverse=True)) 

In [24]:
hypothese = list(snli_data_pd_preprocessed["sentence2_preprocessed"].values)
hypothese

[['person', 'training', 'horse', 'competition'],
 ['person', 'diner', 'ordering', 'omelette'],
 ['person', 'outdoors', 'horse'],
 ['smiling', 'parents'],
 ['children', 'present'],
 ['kids', 'frowning'],
 ['boy', 'skates', 'sidewalk'],
 ['boy', 'skateboarding', 'trick'],
 ['boy', 'wearing', 'safety', 'equipment'],
 ['older', 'man', 'drinks', 'juice', 'waits', 'daughter', 'get', 'work'],
 ['boy', 'flips', 'burger'],
 ['elderly', 'man', 'sits', 'small', 'shop'],
 ['women', 'hugging', 'vacation'],
 ['women', 'sleeping'],
 ['women', 'showing', 'affection'],
 ['people', 'eating', 'omelettes'],
 ['people', 'sitting', 'desks', 'school'],
 ['diners', 'restaurant'],
 ['man', 'drinking', 'juice'],
 ['two', 'women', 'restaurant', 'drinking', 'wine'],
 ['man', 'restaurant', 'waiting', 'meal', 'arrive'],
 ['blond', 'man', 'getting', 'drink', 'water', 'fountain', 'park'],
 ['blond',
  'man',
  'wearing',
  'brown',
  'shirt',
  'reading',
  'book',
  'bench',
  'park'],
 ['blond', 'man', 'drinking', 

In [25]:
co_occurence_two_words(snli_data_pd_preprocessed, "men", "women") 

2292

In [26]:
N = len(snli_data_pd_preprocessed)
N

549526

In [27]:
calculate_pmi(df=snli_data_pd_preprocessed, N=N, counts=sorted_dict_10 , w_i="men", w_j="women")

-0.7356023133153547

In [28]:
sorted_dict_100 = remove_less_than_n_dict(d=sorted_dict, n=100)

In [29]:
men_top = tool(df=snli_data_pd_preprocessed, N=N, counts=sorted_dict_100, w="men")

iter 0 : man, PMI: -3.2385865920726484
iter 1 : woman, PMI: -2.6553466567906883
iter 2 : two, PMI: 1.061879277119484
iter 3 : people, PMI: -3.7909707463173437
iter 4 : wearing, PMI: -0.9188072264204178
iter 5 : young, PMI: -1.2088703873855706
iter 6 : men, PMI: 3.158835804448575
iter 7 : playing, PMI: -0.5372104910824801
iter 8 : girl, PMI: -5.2829751332914405
iter 9 : boy, PMI: -4.7105701297948785
iter 10 : white, PMI: -1.1964448106691468
iter 11 : shirt, PMI: -2.4662530152637507
iter 12 : black, PMI: -1.3334264798543405
iter 13 : dog, PMI: -4.419844776558899
iter 14 : sitting, PMI: -0.6813692405430056
iter 15 : blue, PMI: -1.3839477374633047
iter 16 : standing, PMI: -0.28903588281078374
iter 17 : red, PMI: -1.3955548766299481
iter 18 : group, PMI: -0.03189530857981145
iter 19 : walking, PMI: -1.1599584043556475
iter 20 : outside, PMI: -0.7683631407014834
iter 21 : street, PMI: -0.9469990557432056
iter 22 : person, PMI: -5.87639513266816
iter 23 : front, PMI: -0.6621386500996895
iter 

In [30]:
men_top

{'men': 3.158835804448575,
 'tuxedos': 2.796934730107934,
 'jumpsuits': 2.3529370513793797,
 'ladders': 2.257370075956123,
 'wetsuits': 2.199344621059028,
 'guitars': 2.1676357613316903,
 'harnesses': 2.0856003708635478,
 'vests': 2.0397657171458796,
 'dominoes': 2.03668071819465,
 'fatigues': 1.9382708985011279,
 'suits': 1.889829874813191,
 'hats': 1.8580735212067925,
 'discussion': 1.8455600837076753,
 'ties': 1.7612419615823214,
 'deere': 1.7406958905595693,
 'shaded': 1.6885481389747539,
 'inspect': 1.6623165974015741,
 'shirts': 1.6214070224362453,
 'aprons': 1.6021350600265922,
 'separate': 1.594405063233694,
 'laptops': 1.567811925140032,
 'discuss': 1.56590364160856,
 'microphones': 1.5614930755292464,
 'interior': 1.558353367998679,
 'cannon': 1.548893038749613,
 'guns': 1.5295613885498027,
 'politics': 1.5104188909349772,
 'jackets': 1.4993993434462998,
 'barbecuing': 1.482259340827537,
 'kickboxing': 1.473599814006326,
 'docked': 1.4708905267483399,
 'beers': 1.464377219789

In [33]:
boy_top = tool(df=snli_data_pd_preprocessed, N=N, counts=sorted_dict_100, w="boy")

iter 0 : man, PMI: -3.0717761684290563
iter 1 : woman, PMI: -2.7607527195615575
iter 2 : two, PMI: -3.0571960491224166
iter 3 : people, PMI: -4.300824730668298
iter 4 : wearing, PMI: -0.6623990649253901
iter 5 : young, PMI: 0.8924479347749501
iter 6 : men, PMI: -4.7105701297948785
iter 7 : playing, PMI: -0.5085716793011967
iter 8 : girl, PMI: -0.7859211053355758
iter 9 : boy, PMI: 3.213087248346606
iter 10 : white, PMI: -1.4299757131471944
iter 11 : shirt, PMI: -0.0768228981165908
iter 12 : black, PMI: -1.530230520664385
iter 13 : dog, PMI: -2.9593354724231036
iter 14 : sitting, PMI: -1.208077999173464
iter 15 : blue, PMI: -0.11156522052260921
iter 16 : standing, PMI: -1.172616831012183
iter 17 : red, PMI: -0.3644944426086926
iter 18 : group, PMI: -3.945794902234321
iter 19 : walking, PMI: -1.7374906964124421
iter 20 : outside, PMI: -1.3398169604884123
iter 21 : street, PMI: -2.0308837247485525
iter 22 : person, PMI: -4.058423820797806
iter 23 : front, PMI: -1.5090002198527754
iter 24 

In [34]:
boy_top

{'boy': 3.213087248346606,
 'scouts': 2.7012472718997413,
 'scout': 2.639659053484527,
 'pinata': 2.064322874430033,
 'pumpkins': 1.9612293814659294,
 'trunks': 1.9076904938970145,
 'go-cart': 1.894416029116123,
 'legos': 1.8882263576851672,
 'floaties': 1.8436464022000916,
 'alligator': 1.8427009925719675,
 'teenage': 1.7918070904210222,
 'little': 1.7462720169976076,
 'spiderman': 1.7160234893336586,
 'elmo': 1.652598109192025,
 'bat': 1.579192167465764,
 'cartwheel': 1.5530624356411107,
 'flour': 1.5495046162279213,
 'pigeon': 1.5398301790300752,
 'slide': 1.5032427239132344,
 'zip': 1.4862397227699051,
 'pumpkin': 1.4757203471079456,
 'cheeks': 1.4641530197789145,
 'supported': 1.4440114873806076,
 'patch': 1.4410940039983131,
 'bouncing': 1.4410940039983131,
 'popsicle': 1.4373301222889585,
 'pajamas': 1.43701621997247,
 'grins': 1.4074856113554934,
 'crawls': 1.4040217560519221,
 'pail': 1.4010593421960604,
 'cookie': 1.3860831346232694,
 'bases': 1.3671958903297765,
 'pirate': 1

In [31]:
a

[[1, 2], [2, 5, 6]]

In [32]:
b = tool(df=snli_data_pd_preprocessed, N=N, counts=sorted_dict_10, w="women", limit=200)

TypeError: tool() got an unexpected keyword argument 'limit'

In [ ]:
b

{'women': 3.983784760219459,
 'two': 0.9226235061038228,
 'dancing': 0.904901900424744,
 'walk': 0.8998019130549946,
 'sit': 0.8930064916002304,
 'three': 0.7334620225268811,
 'talking': 0.5331834153633819,
 'together': 0.4912923193555941,
 'four': 0.32196151279458485,
 'table': 0.3011834878530402,
 'asian': 0.16777000472224365,
 'stand': 0.1500867505409876,
 'look': 0.12373851309436806,
 'bench': 0.03310739723648343,
 'store': 0.032337924296353196,
 'one': -0.007869649670569316,
 'smiling': -0.03035222108178292,
 'group': -0.07161765964654923,
 'dressed': -0.0902398826096017,
 'older': -0.09943712593025274,
 'several': -0.11451990865640571,
 'stage': -0.16037394871787525,
 'sidewalk': -0.259474487836655,
 'walking': -0.26339305169628013,
 'blond': -0.31292827627761277,
 'street': -0.35590686276049666,
 'bus': -0.38305592554898416,
 'sitting': -0.42160945572226843,
 'camera': -0.4329080334497591,
 'dark': -0.4419697320043399,
 'carrying': -0.5043878122572143,
 'restaurant': -0.51316895

In [35]:
def write_dict_to_txt(dictionary, filename):
    with open(filename, 'w') as file:
        for key, val in dictionary.items():
            file.write(str(key) + ": \t" + str(val) + '\n')

In [36]:
write_dict_to_txt(men_top, "men.txt")

In [37]:
write_dict_to_txt(boy_top, "boy.txt")

In [55]:
def extract_hypotheses(df:pd.DataFrame, word:str, seed_w:str) -> pd.DataFrame:
    hypo = []
    prem = []
    gold = []
    annotator = []
    for i in range(len(df)):
        if word in df["sentence2_preprocessed"].values[i] and seed_w in df["sentence2_preprocessed"].values[i]:
            hypo.append(df["sentence2"].values[i])
            prem.append(df["sentence1"].values[i])
            gold.append(df["gold_label"].values[i])
            annotator.append(df["annotator_labels"].values[i])
    
    return pd.DataFrame({
        "premise" : prem,
        "hypothese": hypo,
        "gold": gold,
        "annotators": annotator
    })


In [41]:
ladders = extract_hypotheses(df=snli_data_pd_preprocessed, word="ladders", seed_w="men")

In [42]:
ladders

,premise,hypothese,gold,annotators
0,Man wearing jeans and black shirt rides down t...,This is the best way to transport ladders,contradiction,[contradiction]
1,A man standing on two ladders below an attic a...,"Extreme expression on the face of a man, while...",entailment,[entailment]
2,A man standing on two ladders below an attic a...,Man trying to do something while standing on l...,neutral,[neutral]
3,"A man is standing on two ladders, looking scared.",The man is holding tightly to the ladders.,neutral,[neutral]
4,A bearded man standing across two ladders in a...,Man doing something with the ladders,neutral,[neutral]
5,A bearded man standing across two ladders in a...,"Horrified expression on the face of a man, whi...",entailment,[entailment]
6,A woman in a headscarf walks away from a booth...,A woman and two young boys are playing a game ...,contradiction,"[contradiction, contradiction, contradiction, ..."
7,Three individuals in blue shirts work on repai...,Three individuals on ladders repair a ceiling.,neutral,[neutral]
8,A blond woman sitting at a desk in a dimly lit...,The woman is sitting in a dim lighted room and...,entailment,[entailment]
9,A blond woman sitting at a desk in a dimly lit...,A woman sitting at a table in a dimly lit spac...,neutral,[neutral]


In [44]:
ladders.loc[6]["premise"]

'A woman in a headscarf walks away from a booth while holding the hands of two young boys in striped shirts.'

In [45]:
ladders.loc[6]["hypothese"]

'A woman and two young boys are playing a game of shoots and ladders.'

In [46]:
ladders.loc[6]["annotators"]

['contradiction',
 'contradiction',
 'contradiction',
 'contradiction',
 'contradiction']

In [47]:
tuxedos = extract_hypotheses(df=snli_data_pd_preprocessed, word="tuxedos")

In [48]:
tuxedos

,premise,hypothese,gold,annotators
0,People in pirate outfits are playing in a band.,"The band is dressed in tuxedos, having drinks ...",contradiction,[contradiction]
1,A group of men dressed in blue in Monk style r...,The group of men are dressed in tuxedos.,contradiction,[contradiction]
2,Bizarre live art exhibit with goddess type wom...,Three people in an art exhibit are wearing tux...,contradiction,[contradiction]
3,"Three people, wearing orange and black jumpsui...",the people are in blue tuxedos,contradiction,[contradiction]
4,Four people watching a man playing a musical i...,Four people in tuxedos watch a man play guitar.,neutral,[neutral]
5,Woman poses playfully in white dress.,There is a group of men in tuxedos posing.,contradiction,[contradiction]
6,"Two men, one wearing jeans and a green shirt a...",There are two men in tuxedos.,contradiction,[contradiction]
7,A group of well dressed men with a pack of dogs.,A group of men are wearing tuxedos,neutral,[neutral]
8,Five men in tuxedos are walking down steps.,Five men in multi colored tuxedos are walking...,neutral,[neutral]
9,Five men in tuxedos are walking down steps.,Five men in tuxedos are walking down steps.,entailment,[entailment]


In [51]:
def print_infos(df:pd.DataFrame, index:int) -> None:
    print("premise: \t", df.loc[index]["premise"])
    print("hypothese: \t", df.loc[index]["hypothese"])
    print("gold: \t", df.loc[index]["gold"])
    print("annotator: \t", df.loc[index]["annotators"])

In [52]:
print_infos(df=tuxedos, index=22)

premise: 	 3 men in tuxedos and 2 women in dresses, sleeping on the grass.
hypothese: 	 The men in tuxedos and women in dresses are sleeping in the grass.
gold: 	 entailment
annotator: 	 ['entailment']


In [53]:
jumpsuits = extract_hypotheses(df=snli_data_pd_preprocessed, word="jumpsuits")
jumpsuits

,premise,hypothese,gold,annotators
0,Aid big guy wearing an orange jumpsuit is talk...,Orange jumpsuits are conspicuously visible.,entailment,[entailment]
1,People are walking by a food cart in the city.,People in red jumpsuits jump over sleeping kids.,contradiction,[contradiction]
2,Several men dressed in back jumpsuits sing and...,Humans dressed in jumpsuits.,entailment,[entailment]
3,Several men dressed in back jumpsuits sing and...,Tall humans in jumpsuits.,neutral,[neutral]
4,Several men dressed in back jumpsuits sing and...,Nobody in jumpsuits.,contradiction,[contradiction]
5,Two men in white and orange jumpsuits skydiving.,The two men in white and orange jumpsuits skyd...,neutral,[neutral]
6,Two men in white and orange jumpsuits skydiving.,Two men in white and orange jumpsuits skydived...,entailment,[entailment]
7,Two men in white and orange jumpsuits skydiving.,Two men in white and orange jumpsuits paraglid...,contradiction,[contradiction]
8,Three men walk across a street in black unifor...,A manager watches three construction workers i...,neutral,[neutral]
9,Two Rubbish men standing on their truck loadin...,People wearing jumpsuits enjoying the breeze o...,neutral,[neutral]


In [54]:
print_infos(df=jumpsuits, index=0)

premise: 	 Aid big guy wearing an orange jumpsuit is talking to a skinny lady who's dressed up with a long skirt is standing on the freeway looking at the red Motorcycle that crash
hypothese: 	 Orange jumpsuits are conspicuously visible.
gold: 	 entailment
annotator: 	 ['entailment']
